# **1. Starting Setup**




## 1.1 Download dependencies

In [1]:
!pip install -q "transformers>=4.40.0" datasets accelerate bitsandbytes sentencepiece protobuf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.0 MB/s eta 0:00:00


In [2]:
import torch
import re
from tqdm import tqdm

## 1.2 Setting runtime

In [3]:
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("⚠️ No GPU. Go to Runtime → Change runtime type → set GPU.")

CUDA available: True
GPU: Tesla T4


## 1.3 Import Dataset

In [4]:
!git clone https://github.com/sooo66/semeval2026-task12-dataset.git
!ls semeval2026-task12-dataset

Cloning into 'semeval2026-task12-dataset'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 22 (delta 6), reused 16 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 4.25 MiB | 17.12 MiB/s, done.
Resolving deltas: 100% (6/6), done.
dev_data  README.md  sample_data  train_data


## 1.4 Constants definition

In [5]:
from pathlib import Path
import json
from typing import Dict, List, Set

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Root of the cloned repo in Colab
DATA_ROOT = Path("/content/semeval2026-task12-dataset")

# Choose the split you want to evaluate: "train_data", "dev_data", or "sample_data"
SPLIT = "dev_data"  # you can change this

QUESTIONS_FILE = DATA_ROOT / SPLIT / "questions.jsonl"
DOCS_FILE      = DATA_ROOT / SPLIT / "docs.json"

print("Questions file:", QUESTIONS_FILE)
print("Docs file:", DOCS_FILE)

Questions file: /content/semeval2026-task12-dataset/dev_data/questions.jsonl
Docs file: /content/semeval2026-task12-dataset/dev_data/docs.json


# **2. Functions to load dataset**

In [6]:
def load_jsonl(path: Path) -> List[Dict]:
    """Read a .jsonl file into a list of dicts."""
    items = []
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            items.append(json.loads(line))
    return items

def load_json(path: Path):
  """Read a .json file."""
  with path.open("r", encoding="utf-8") as f:
      return json.load(f)

def index_docs_by_topic(docs_list: List[Dict]) -> Dict[int, List[Dict]]:
    """
    Build a mapping: topic_id -> docs (list of dicts).
    Each item in docs_list has: {"topic_id": int, "docs": [ { ... }, ... ] }
    """
    result: Dict[int, List[Dict]] = {}
    for d in docs_list:
        tid = d["topic_id"]
        result[tid] = d["docs"]
    return result

questions = load_jsonl(QUESTIONS_FILE)
docs = load_json(DOCS_FILE)
docs_by_topic = index_docs_by_topic(docs)

print("Num questions:", len(questions))
print("Num doc groups:", len(docs_by_topic))
print("Example question keys:", questions[0].keys())

Num questions: 400
Num doc groups: 36
Example question keys: dict_keys(['topic_id', 'uuid', 'target_event', 'option_A', 'option_B', 'option_C', 'option_D', 'golden_answer'])


# **3. Loading Model**



In [7]:
MODEL_NAME = "google/gemma-2-9b-it"

# 1. Definition of the quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,              # Active 4-bit loading
    bnb_4bit_quant_type="nf4",      # Use the type "nf4" (more precise)
    bnb_4bit_compute_dtype=torch.float16  # loat16
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True
)

# 2. Loading model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)

model.eval()
print("Model loaded on:", model.device)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Model loaded on: cuda:0


# **4. Prompt Generation**

## 4.1 Build Context

In [8]:
def build_context(docs, max_total_chars):
  if not docs:
        context_text = "No context document available"
  else:
        full_context_list = []
        current_length = 0

        for i, doc in enumerate(docs, 1):
            # Take snippet or content
            raw_text = doc.get('content') or doc.get('snippet', '')

            # Clear text to remove useless blank spaces (to reduce waste of resources)
            clean_text = " ".join(raw_text.split())

            # Create the document string
            doc_str = f"--- Document {i} ---\nTitle: {doc.get('title')}\nText: {clean_text}\n\n"

            # Check length: if adding this doc we exceed the limit we don't add it
            if current_length + len(doc_str) > max_total_chars:
                # Add only the part that stays in the limit
                remaining = max_total_chars - current_length
                full_context_list.append(doc_str[:remaining] + "... [TRUNCATED]")
                break
            full_context_list.append(doc_str)
            current_length += len(doc_str)

        context_text = "".join(full_context_list)

## 4.2 Prompt Structure

In [9]:
SYSTEM_PROMPT = (
    "You are solving SemEval 2026 Task 12: Abductive Event Reasoning. "
    "Given an event, context documents, and four options (A–D), "
    "choose which option(s) are the most plausible direct cause of the event. "
    "Respond ONLY with the letters of all correct options, "
    "separated by commas (e.g. 'A', 'A,B', or 'D'). "
    "Do not output any explanations."
)

def format_gemma_prompt(question, docs, max_total_chars=6000):
  #1 Obtain the context
  context_text = build_context(docs,max_total_chars)

  # 2. Prompt Composition
  user_message = f"""{SYSTEM_PROMPT}

Event: "{question['target_event']}"

Context Documents:
{context_text}

Options:
A) {question['option_A']}
B) {question['option_B']}
C) {question['option_C']}
D) {question['option_D']}

Answer (letters only):"""

  messages = [{"role": "user", "content": user_message}]
  prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
  return prompt

# **5. Output parsing and scoring function**

In [10]:
def clean_response(response_text):
    """
    Clean model output to extract only the letters A, B, C, D.
    Handle cases like "Option A", "A.", "A, B", "The answer is C".
    """
    # Remove everything except letters and commas
    # Find for pattern like A, B, C, D
    # Find all uppercase letters A-D in the text
    matches = re.findall(r'\b[A-D]\b', response_text.upper())

    # If does not find anything, return empty string
    if not matches:
        return set()

    # Return set for easy comparision (the order does not count for sets)
    return set(matches)


def calculate_score(prediction_set, golden_string):
    """
    Calculate score: 1.0 (Exact), 0.5 (Partial), 0.0 (Wrong)
    Exact: All letters are correct
    Partial: Some letters are contained in the correct set, but not all
    Wrong: There is a letter not contained in the correct set
    """
    # Clean also the golden answer (that arrives as a string: "C" o "A,B")
    golden_set = set(re.findall(r'\b[A-D]\b', golden_string.upper()))

    if not golden_set:
        print(f"Warning: Golden answer empty or malformed: {golden_string}")
        return 0.0

    # Case 1: Equal (Exact) -> 1 point
    if prediction_set == golden_set:
        return 1.0

    # Case 2: Golden Answer CONTAIN the prediction -> 0.5 points
    # Example: Gold={A, B}, Pred={A} -> Gold contains Pred
    # Esempio: Gold={A}, Pred={A, B} -> Gold DOES NOT contain Pred
    elif prediction_set.issubset(golden_set):
        return 0.5

    # Case 3: Golden does not contain predicted (Wrong) -> 0 punti
    else:
        return 0.0

# **6. Evaluation loop**




## 6.1 Function to compute metrics over the results

In [26]:
def print_metrics(results):
  perfect_answers = [r for r in results if r['score'] == 1.0]
  partial_answers = [r for r in results if r['score'] == 0.5]

  n_total = len(results)
  n_correct = len(perfect_answers)
  n_partial = len(partial_answers)
  n_wrong = n_total - n_correct - n_partial

  total_score = sum(r['score'] for r in results)

  print(f"======= {MODEL_NAME} Causal Reasoning Results =======")
  print(f"Total questions: {n_total}")
  print(f"Correct answers: {n_correct} - {n_correct/n_total*100}%")
  print(f"Partial answers: {n_partial} - {n_partial/n_total*100}%")
  print(f"Wrong answers: {n_wrong} - {n_wrong/n_total*100}%")
  print(f"Total score: {total_score}")
  print(f"Performance of the score: {total_score/n_total*100}%")


## 6.2 Checkpoints loading

### Mount The Drive and set the Checkpoint path

In [12]:
import json
from google.colab import drive
import os

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

CHECKPOINT_FILE = "/content/drive/MyDrive/LLM/Gemma/evaluation_checkpoint.jsonl"

Mounted at /content/drive


### Remove previous checkpoints

**CAREFUL: RUN THIS SNIPPET ONLY IF YOU WANT TO START FROM THE BEGINNING**

In [13]:
## !!! This will delete the checkpoints in your drive !!!
## Commented so that we can use the "Run all previous"

# !rm -rf $CHECKPOINT_FILE

### Load the checkpoints if it exists

In [16]:
results = []

processed_uuids = set() # Set to check which question to skip (already processed)

if os.path.exists(CHECKPOINT_FILE):
    print(f"📂 Found checkpoint: {CHECKPOINT_FILE}. Loading checkpoints...")
    with open(CHECKPOINT_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                try:
                    data = json.loads(line)
                    results.append(data)
                    processed_uuids.add(data['uuid']) # Memorize processed ID
                    total_score += data['score']
                except json.JSONDecodeError:
                    continue
    print(f"Restored {len(results)} previous results.")
else:
    print("No checkpoint found. Start a new evaluation")

📂 Found checkpoint: /content/drive/MyDrive/LLM/Gemma/evaluation_checkpoint.jsonl. Loading checkpoints...
Restored 33 previous results.


## 6.3 Evaluation Loop

In [17]:
import gc   # garbage collector

total_score = 0
count = 0
errors = 0

print(f"Starting evaluation on {len(questions)} questions...")


# we use tqdm to see the progress bar
for entry in tqdm(questions):
    question_uuid = entry['uuid']

    # --- SKIP LOGIC ---

    # If UUID is in the processed sed, we skip the iteration
    if question_uuid in processed_uuids:
        continue

    # ------------------

    try:
        topic_id = entry['topic_id']
        question_uuid = entry['uuid']
        golden_ans = entry['golden_answer']

        # Recover documents for this topic
        context_docs = docs_by_topic.get(topic_id, [])

        # 1. Create the prompt
        prompt = format_gemma_prompt(entry, context_docs, max_total_chars=6000)

        # 2. Tokenization
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        # 3. Generation (Greedy decoding for reproducibility)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=15,    # Enough for "A, B, C"
                do_sample=False,      # Determinism
                temperature=0.0,
                pad_token_id=tokenizer.eos_token_id
            )

        # 4. Decoding e Cleaning
        # Cutting the input prompt -> the response of the model also contains the input prompt we provided
        generated_ids = outputs[0][inputs.input_ids.shape[1]:]
        raw_response = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

        # Extract the letters set (es. {"A", "B"})
        pred_set = clean_response(raw_response)

        # 5. Calculate score
        score = calculate_score(pred_set, golden_ans)

        # We prepare the result object (a list of result_item, which is the following dictionare)
        result_item = {
            "uuid": question_uuid,
            "topic_id": topic_id,
            "golden_raw": golden_ans,
            "prediction_raw": raw_response,
            "prediction_set": list(pred_set),
            "score": score
        }

        # --- IMMEDIATE SAVING (Checkpoint) ---
        # Append (mode 'a') a new line to the JSONL file
        # Basically, we save in a json (on the drive) every computed answer,
        # so that if something goes wrong we can restore it from the drive
        with open(CHECKPOINT_FILE, 'a', encoding='utf-8') as f:
            f.write(json.dumps(result_item) + "\n")

        # Updating variables in memory
        results.append(result_item)
        processed_uuids.add(question_uuid)
        total_score += score
        count += 1

    except RuntimeError as e:
        if "out of memory" in str(e):
            print(f"UUID:{entry['uuid']} process skipped due to OOM")
            # forcely free the cache
            del inputs
            torch.cuda.empty_cache()
            gc.collect()
            errors += 1
            continue
        else:
            print(f"Errore generico: {e}")
            continue

Starting evaluation on 400 questions...


100%|██████████| 400/400 [06:23<00:00,  1.04it/s]


# **7. Results**

In [27]:
# Final Report -- Printing Correct, Partial, Wrong answers and the score

print_metrics(results)

======= google/gemma-2-9b-it Causal Reasoning Results =======
Total questions: 400
Correct answers: 180 - 45.0%
Partial answers: 90 - 22.5%
Wrong answers: 130 - 32.5%
Total score: 225.0
Performance of the score: 56.25%
